In [1]:
import pandas as pd
from sklearn import model_selection, metrics, ensemble

In [2]:
df_train = pd.read_csv('../../Feature_Encoding/data/train_count_encoding.csv')
df_test = pd.read_csv('../../Feature_Encoding/data/test_count_encoding.csv')
train = pd.read_csv('../../Feature_Engineering/data/other-cleaned_train.csv')

In [3]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [4]:
#Realiza busqueda completa combinando los parametros
def bagging(x_train, y_train, x_validation, y_validation):
    bag_classifier = ensemble.BaggingClassifier()
    params_bag = {'n_estimators':[1,2,3,4,5,6,7,8,9,10,15,25],'max_samples':[1.0],'max_features':[1.0]}    
    bag_gs = model_selection.GridSearchCV(bag_classifier, params_bag, cv=3)
    bag_gs.fit(x_train, y_train)
    bag_best = bag_gs.best_estimator_
    print(bag_gs.best_params_)
    print('bag: {}'.format(bag_best.score(x_validation, y_validation)))
    return bag_best

In [5]:
def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = metrics.log_loss(y_test, predictions)
    accuracy = metrics.accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))


In [6]:
y = train.Target
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(df_train, y, test_size=0.3, stratify=y)

In [7]:
bagging = bagging(x_train, y_train, x_validation, y_validation)
test_model(bagging, x_validation, y_validation)
cross_val(bagging, x_train, y_train)

{'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 25}
bag: 0.9190523198420533
Accuracy: 91.91%, Logloss: 0.26
0.901506158465285


In [8]:
for column in df_test.columns.tolist():
    df_test[column] = df_test[column].fillna(0)   

In [9]:
y_pred = bagging.predict_proba(df_test)[:,1]
submission = pd.DataFrame(data={'Opportunity_ID':df_test['Opportunity_ID'], 'Target': y_pred})
submission = submission.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission.to_csv('../submits/count_bag.csv', index=False)